## Character Level Language Model
predicting next character in the sequence, given some characters before it


In [1]:
words = open('names.txt', 'r').read().splitlines()    #To get our words as a list

In [2]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [3]:
len(words) #total no of words

32033

In [4]:
min(len(w) for w in words) #shortest word

2

In [5]:
max(len(w) for w in words) # longest wprd

15

### Bigram Language Model
  Two characters at a time/ Two characters in a row

Inorder to learn the statistics of which characters are likely to follow other characters, the simplest way is to do it by counting, we're going to count how often any one of these combinations occurs in the training set

In [6]:
b = {}                                   # dictionary that will maintain some counts for every one of these bigrams
for w in words:                          #each w is an individual word, we wannna iterate this word with consecutive characters
    chs = ['<S>'] + list(w) + ['<E>']    #'<S> and <E> Shows us Start and End character. list will let us have individual characters  i.e ['e', 'm', 'm', 'a']
    for ch1, ch2 in zip(chs, chs[1:]):   #we zip a word with itself ch1 = emma, ch2 = mma zip(ch1, ch2) = (em), (mm), (ma)
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1  # this will add up all the bigrams and count how often they occur i.e <E> 'a' occurs 3 times in first 3 words
        #print(ch1, ch2)

The [zip()](https://www.w3schools.com/python/ref_func_zip.asp) function returns a zip object, which is an iterator of tuples where the first item in each passed iterator is paired together, and then the second item in each passed iterator are paired together etc.

    a = ("John", "Charles", "Mike")
    b = ("Jenny", "Christy", "Monica")

    x = zip(a, b)     print(x)

    (('John', 'Jenny'), ('Charles', 'Christy'), ('Mike', 'Monica'))

In [7]:
list(w)

['z', 'z', 'y', 'z', 'x']

In [7]:
sorted(b.items(), key = lambda kv: -kv[1])      # sort by the count of these elements

[(('n', '<E>'), 6763),
 (('a', '<E>'), 6640),
 (('a', 'n'), 5438),
 (('<S>', 'a'), 4410),
 (('e', '<E>'), 3983),
 (('a', 'r'), 3264),
 (('e', 'l'), 3248),
 (('r', 'i'), 3033),
 (('n', 'a'), 2977),
 (('<S>', 'k'), 2963),
 (('l', 'e'), 2921),
 (('e', 'n'), 2675),
 (('l', 'a'), 2623),
 (('m', 'a'), 2590),
 (('<S>', 'm'), 2538),
 (('a', 'l'), 2528),
 (('i', '<E>'), 2489),
 (('l', 'i'), 2480),
 (('i', 'a'), 2445),
 (('<S>', 'j'), 2422),
 (('o', 'n'), 2411),
 (('h', '<E>'), 2409),
 (('r', 'a'), 2356),
 (('a', 'h'), 2332),
 (('h', 'a'), 2244),
 (('y', 'a'), 2143),
 (('i', 'n'), 2126),
 (('<S>', 's'), 2055),
 (('a', 'y'), 2050),
 (('y', '<E>'), 2007),
 (('e', 'r'), 1958),
 (('n', 'n'), 1906),
 (('y', 'n'), 1826),
 (('k', 'a'), 1731),
 (('n', 'i'), 1725),
 (('r', 'e'), 1697),
 (('<S>', 'd'), 1690),
 (('i', 'e'), 1653),
 (('a', 'i'), 1650),
 (('<S>', 'r'), 1639),
 (('a', 'm'), 1634),
 (('l', 'y'), 1588),
 (('<S>', 'l'), 1572),
 (('<S>', 'c'), 1542),
 (('<S>', 'e'), 1531),
 (('j', 'a'), 1473),
 (

('n', '< e >'), 6763) tells us that n was the last character 6763 times

(('< S >', 'a'), 4410) tells us that a was the first character 4410 times

(('a', 'n'), 5438), tells us a was followed by n 5438 times

Store this info in a 2d array. The rows are going to be the first character of the bigram and the columns are going to be the second character. 
    And each entry in this 2d array will tell us how often the second character follows the first character in the dataset
    
    The array library that we'll use is going to use is pytorch

In [8]:
import torch

create an array of zeros and we give it a size of array, eg 3 by 5 array of zeros

by default you will notice the data type is float32 (a.dtype), because we're going to represent counts. use dtype as torch.in32

Tensors allow us to manipulate all individual entries efficiently eg change value at row 2 column 4 a[ 1,3 ] = 1

In our case we have 26 letters of the alphabet and we have tro special characters < s> and < e>. so we want a 28 by 28 array, and let's call it N because it is going to represent counts

In [15]:
N = torch.zeros(28, 28, dtype = torch.int32)

Because our characters are strings, we have to index into an array and we have to  index using integers, so we need some kind of a lookup table from characters to integers

We're going to  take all the words which is a list of strings. we're going to concantenate all of it into a massive string (simply the entire dataset as a single string). 

We're going to pass this to the set constructor, which takes this massive string and throws out duplicates because sets do not allow duplicates. so set of words will just be a list of all lowercase characters set(''.join(words))

then make it a list and sort it, and call it characters chars = ist(set(''.join(words))))

Now what we want is this lookup table mentioned above, so let's create a special stoi, and this will be an s:i mapping. enumarate basicallly gives us this iterator over the integer index and the actual element of the list. And then we're mapping the character to the integer

so a = 0, b = 1, c = 2 .......

but we have to specify that < S> = 26 and < E> = 27 because z was 25

In [26]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i for i,s in enumerate(chars)}
stoi['<S>'] = 26
stoi['<E>'] = 27

In [27]:
stoi

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25,
 '<S>': 26,
 '<E>': 27}

In [29]:
# this should give us a large 28 x 28 array of all these counts we just made
for w in words:                          #each w is an individual word, we wannna iterate this word with consecutive characters
    chs = ['<S>'] + list(w) + ['<E>']    #'<S> and <E> Shows us Start and End character. list will let us have individual characters  i.e ['e', 'm', 'm', 'a']
    for ch1, ch2 in zip(chs, chs[1:]):   #we zip a word with itself ch1 = emma, ch2 = mma zip(ch1, ch2) = (em), (mm), (ma)
        ix1 = stoi[ch1]                  # map character 1 and 2 to their integers that we just did above (how we connected alphabets to their arrays)
        ix2 = stoi[ch2] 
        N[ix1, ix2] += 1                 # 2d array indexing (manipulating the array and adding one, because it starts at zero i.e a = 0)

Now it looks like an ugly mess. SO lets try to visualize it look more nicer 18:24